# Homework nr. 2 - data cleaning (deadline 8/11/2018)

In short, the main task is to clean The Metropolitan Museum of Art Open Access dataset.
  
> The instructions are not given in details: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can. ;)

## What are you supposed to do:

  1. Download the dataset MetObjects.csv from the repository https://github.com/metmuseum/openaccess/.
  2. Check consistency of at least three features where you expect problems (include "Object Name" feature).
  3. Select some features where you expect integrity problems (describe your choice) and check integrity of those features.
  4. Convert at least five features to a proper data type. Choose at least one numeric, one categorical and one datetime.
  5. Find some outliers (describe your selection criteria).
  6. Detect missing data in at least three features, convert them to a proper representation (if they are already not), and impute missing values in at least one feature.

**If you do all this properly, you will obtain 6 points**

To earn **extra two points** you can do some of these:
  * Focus more precisely on cleaning of the "Medium" feature. Such if you like to use it in KNN based algorithms later.
  * Focus on the extraction of physical dimensions of each item (width, depth and height in centimeters) from the "Dimensions" feature.

## Comments

  * Please follow the instructions from https://courses.fit.cvut.cz/MI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, he can give you another chance to rework your homework and to obtain more points.

In [24]:
import pandas as pd
import numpy as np
import sklearn as skit
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
np.set_printoptions(threshold=np.nan)

forbidden_columns = []
df = pd.read_csv('MetObjects.csv',sep=',',quotechar='"')
print('LOADING FINISHED')
df.info();

LOADING FINISHED
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472255 entries, 0 to 472254
Data columns (total 43 columns):
Object Number              472255 non-null object
Is Highlight               472255 non-null bool
Is Public Domain           472255 non-null bool
Object ID                  472255 non-null int64
Department                 472255 non-null object
Object Name                468278 non-null object
Title                      441187 non-null object
Culture                    192212 non-null object
Period                     73154 non-null object
Dynasty                    23305 non-null object
Reign                      11212 non-null object
Portfolio                  21840 non-null object
Artist Role                281421 non-null object
Artist Prefix              96648 non-null object
Artist Display Name        282640 non-null object
Artist Display Bio         235157 non-null object
Artist Suffix              10746 non-null object
Artist Alpha Sort          282611

## Convert at least five features to proper data type 

In [25]:
df['Metadata Date'] = pd.to_datetime(df['Metadata Date'],format="%m/%d/%Y %I:%M:%S %p")
df['Object Begin Date'] = pd.to_numeric(df['Object Begin Date'])
df['Object End Date'] = pd.to_numeric(df['Object End Date'])
    
repository_category = pd.api.types.CategoricalDtype(categories=["Metropolitan Museum of Art, New York, NY"], ordered=False)
if not pd.api.types.is_categorical_dtype(df['Repository']):
    df['Repository'] = df['Repository'].astype(repository_category)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472255 entries, 0 to 472254
Data columns (total 43 columns):
Object Number              472255 non-null object
Is Highlight               472255 non-null bool
Is Public Domain           472255 non-null bool
Object ID                  472255 non-null int64
Department                 472255 non-null object
Object Name                468278 non-null object
Title                      441187 non-null object
Culture                    192212 non-null object
Period                     73154 non-null object
Dynasty                    23305 non-null object
Reign                      11212 non-null object
Portfolio                  21840 non-null object
Artist Role                281421 non-null object
Artist Prefix              96648 non-null object
Artist Display Name        282640 non-null object
Artist Display Bio         235157 non-null object
Artist Suffix              10746 non-null object
Artist Alpha Sort          282611 non-null object


## Replacing missing values

In [26]:
df['Dimensions'].replace(['Dimensions unavailable'], np.nan, inplace=True)

# Consistency
## LowerCaseTest

In [ ]:
#columns = ['Medium']
except_forbidden_columns = df.drop(forbidden_columns, axis=1, inplace=False).columns

columns = np.intersect1d(df.select_dtypes(include='object').columns.values,except_forbidden_columns)

print('Consistency checks:')
print('\tLowerCaseTest:')
for column in columns:
    length_before = df[column].nunique()
    temp = df[column].str.lower()
    length_after = temp.nunique()
    testOutput = 'OK'
    if length_before > length_after:
        df[column+"_original"] = df[column]
        df[column] = temp
        testOutput = 'FAIL'
        forbidden_columns.append(column+'_original')
    print('\t\t'+testOutput+": "+column+" [before: "+str(length_before)+", after: "+str(length_after)+"]");

Consistency checks:
	LowerCaseTest:
		FAIL: Artist Alpha Sort [before: 60809, after: 60790]
		OK: Artist Begin Date [before: 23715, after: 23715]
		FAIL: Artist Display Bio [before: 44842, after: 44836]
		FAIL: Artist Display Name [before: 60803, after: 60798]
		OK: Artist End Date [before: 24039, after: 24039]
		FAIL: Artist Nationality [before: 3806, after: 3803]
		FAIL: Artist Prefix [before: 5530, after: 5365]
		OK: Artist Role [before: 6254, after: 6254]
		FAIL: Artist Suffix [before: 1700, after: 1679]
		FAIL: City [before: 2664, after: 2650]
		OK: Classification [before: 1211, after: 1211]
		FAIL: Country [before: 1090, after: 1070]
		FAIL: County [before: 1161, after: 1142]
		FAIL: Credit Line [before: 38696, after: 38431]
		FAIL: Culture [before: 7185, after: 7126]
		OK: Department [before: 19, after: 19]
		FAIL: Dimensions [before: 253708, after: 252301]
		FAIL: Dynasty [before: 378, after: 364]
		FAIL: Excavation [before: 361, after: 360]
		OK: Geography Type [before: 132, a

## DuplicateRowsTest

In [ ]:
forbidden_columns = forbidden_columns+['Object ID', 'Metadata Date', 'Repository', 'Link Resource', 'Rights and Reproduction']
except_forbidden_columns = df.drop(forbidden_columns, axis=1, inplace=False).columns
#print(forbidden_columns)
temp = df.drop_duplicates(subset=except_forbidden_columns)
length_before = len(df.index)
length_after = len(temp.index)
if length_before > length_after:
    print('DUPLICATE ROWS FOUND[before: '+str(length_before)+', after:'+str(length_after)+']: '+str(length_before-length_after)+" duplicate rows")
    df = temp
    print("Replacing original dataframe with the dataframe without duplicates")
else:
    print('NO DUPLICATES FOUND')

# Integrity checks

## Object Begin Year is not a higher value than Object End Year
It is obviously not correct when an object was created after its finish.

It can be seen that 198 objects have wrongly filled dates. These dates could be derivated from the Object Date column, but the structure of this column has no followed format. The columns can't be repaired with any simple algorithm. So I just add new column, which shows if this integrity check failed or not.

The similar approche could be applied on columns 'Artist End Date' and 'Artist Begin Date'

In [ ]:
forbidden_columns = forbidden_columns + ['integrity_object_date_check']
df['integrity_object_date_check'] = df['Object Begin Date'] < df['Object End Date']

In [ ]:
df[['Object Begin Date', 'Object End Date']]

## Object Begin Date and Object End Date cannot be higher than current year

It can be seen from df.describe() that Object Begin Date and Object End Date has weirdly looking maximal and minimal values, so I replace them with NaN.

In [ ]:
display(df.describe())

display(df[df['Object Begin Date'] > 2018])

df.loc[df['Object Begin Date'] > 2018, 'Object Begin Date']=np.nan
df.loc[df['Object End Date'] > 2018, 'Object End Date']=np.nan